#### Step 1 : Import Required Libs

In [1]:
## Import required Libs
import boto3
import json
from botocore.exceptions import BotoCoreError, ClientError


#### Step 2 : Set AWS Region

In [2]:
## Set AWS Region
try:
    # Set AWS Region
    region = "us-east-1"  # Note the correct region code "us-east-1"
    
    # The Bedrock runtime must be initialized with your AWS credentials/profile and region to authenticate you 
    # and direct requests to the right foundation models or agent.
    bedrock_runtime = boto3.client("bedrock-runtime", region_name=region)
    s3 = boto3.client("s3", region_name=region)
    
    print("Project Environment is setup successfully")
    
except (BotoCoreError, ClientError) as error:
    print(f"An error occurred while setting up AWS clients: {error}")


Project Environment is setup successfully


#### Step-3 : Setup RAG Function

###### 1. Take user query
###### 2. Retrieve relevant information from our knowledge base
###### 3. Generate a response using a foundation model

In [3]:
##--------------------------------------------------------------------------------------------------------------------------------------------- 
## 1] RAG Model - Amazon Titan text foundation model available via Amazon Bedrock.
##  This model is a large language model (LLM) optimized for a wide range of advanced, general-purpose language
##  tasks such as open-ended text generation, conversational chat, and also supports Retrieval Augmented Generation (RAG) workflows.
##  ---
## 2] knowledge_base_id identifies the knowledge base to query.This is a unique identifier (typically a string or an Amazon Resource Name, ARN)
##  that specifies which knowledge base your application or model should access during information retrieval.
##  In the context of Amazon Bedrock: A knowledge base is a structured repository of information—often created by ingesting private documents,
##  databases, or other content. Amazon Bedrock uses these knowledge bases to support Retrieval-Augmented Generation (RAG) workflows.
##  The knowledge_base_id is the unique ID assigned to a specific knowledge base within Amazon Bedrock. When issuing a query,
##  this ID tells Bedrock which knowledge base to search for relevant information.
##---------------------------------------------------------------------------------------------------------------------------------------------
##
model_id = "amazon.titan-text-express-v1"
def generate_rag_response(query,knowledge_base_id, model_id = model_id):
    """ Generate Response using RAG Approach
        Args :
         1] query (str) : The user's question
         2] knowledge_base_id (str) : ID of the knowledge base to query
    """
    try: 
        """ Step-1 : Retrieve information from the knowledge base
        # Note : In a full implemenation, we would call the Bedrock Knowledge Base API here
        # For this example, we will simulate the retrieval with a placeholder
        # bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name = region)
        # kb_response = bedrock_agent_runtime.retrieve(
           knowledgeBaseId=knowledge_base_id,
           retrivalQuery ={'text' : query},
           numberOfResults = 3)
       """
       ## For simplified example:
        retrieved_info =""" Employee Handbook Section 3.2 : Pait Time Off
         All Employees accrue PTO at a rate of 1.5 days per month (18 days per year).
         PTO requests must be submitted at least two weeks in advance through the HR portal
         Unused PTO can be carried over to the following year, up to a maximum of 5 days
         For further queries on PTO, please contact hr@example.com

         Remote work policy for full time employees only - Full time employees are eligible to work remotely 
         up to 2 days per week and max of 10 days per month under unavoidable circumstances only when manager 
         has approved additional remote work days. Under normal circumstances, full time employees are expected to work from office for 3 days a week.
         For consitency, co-ordination and efficiency purpose, all employees are expected to work from
         office from Monday to Wednesday.
                                          """
        
        # Step 2 - Construct the prompt for the model using retrieved information and query
        prompt = f"""
          You are an HR assistant for our company.Use only following information to answer the query.
          If you do not know the answer based on this information then say No - don't make up an answer

          RETRIEVED INFORMATION :
          {retrieved_info}

          USER QUESTION:
          {query}
            """
    
        # Step 3 - Construct the payload
        # Generate a response using the specified model
        if 'claude' in model_id.lower():
            payload = {
               "anthropic_version": "bedrock-2023-05-31", # Specify the version of the Anthropic model to use
               "max_tokens": 512, # How many words (max tokens) the AI can respond with (up to 512).
               "temperature" : 0.5, #How “creative” or random the answers should be (temperature 0.5 means moderately creative).
               "messages" :[
                    {
                         "role": "user",# The role of the user in the conversation
                         "content": prompt # The actual text of the user message, which includes the retrieved information and the user's question
                     
                  }
                  
               ]
            }
        else:
            # Generic format for other models.
            payload = {
                "inputText": prompt,
                "textGenerationConfig": {
                    "maxTokenCount": 256,
                    "temperature": 0.9
                }
            }
        # Step 4 - Call the model   
        response = bedrock_runtime.invoke_model(
            modelId=model_id,
            body=json.dumps(payload), # Convert the payload to JSON format
            contentType="application/json", # Specify the content type as JSON
            accept="application/json" # Specify the response format as JSON
        )
        # Step 5 - Parse the response based on the model type.
        response_body = json.loads(response['body'].read().decode('utf-8')) # Read and decode the response body
        ###       
        if 'claude' in model_id.lower():
            generated_text = response_body["content"][0]["text"]
        else:
            generated_text = response_body["results"][0]["outputText"]
        
        return generated_text
    
    except Exception as e:
        return f"An error occurred while generating the response:{str (e)}"


#### Step 4:  Test RAG Application

In [4]:
# Define a sample knowledge base ID - This would be a real ID in a production scenario
sample_kb_id = "sample hr knowledge base id"  # Replace with your actual knowledge base ID
# Test query
# test_query = "How many PTO days do full-time employees get per year?"
#test_query = "Is current remote work policy is applicable for full time employees only? What about part time employees?"
test_query = " What is the role of my manager ?"
# test_query = "Why all employees are expected to work from office from Monday to Wednesday?"  # Uncomment to use this query
# # Call the function to generate a response
response = generate_rag_response(test_query, sample_kb_id)
print("\n User Query:", test_query)
print("\n Response from RAG Model:", response)
# --- End  ---




 User Query:  What is the role of my manager ?

 Response from RAG Model: 
No


In [5]:
# Test with a question that should be answerable from our knowledge base
test_query_2 = "What's the process for requesting time off?"
response_2 = generate_rag_response(test_query_2, sample_kb_id)

# Test with a question that might not be in our knowledge base
test_query_3 = "What's the company policy on remote work?"
response_3 = generate_rag_response(test_query_3, sample_kb_id)

print("\nQuestion about time off process:")
print(response_2)

print("\nQuestion about remote work policy (not in our example knowledge base):")
print(response_3)



Question about time off process:

You must submit the request at least two weeks in advance via the HR portal.

Question about remote work policy (not in our example knowledge base):

No


In [18]:
# Set up the Knowledge Base Using Vector Database
# Step 1 - Create S3 bucket. Create markdown file containing HR Policies and store in AWS s3

import boto3

# Initialize boto3 S3 client
s3 = boto3.client('s3')

# Generate a unique bucket name using your account ID
account_id = boto3.client('sts').get_caller_identity()['Account']
bucket_name = f"hr-knowledge-base-v02-{account_id}"

# Check if the bucket already exists
def bucket_exists(bucket_name):
    response = s3.list_buckets()
    buckets = [bucket['Name'] for bucket in response['Buckets']]
    return bucket_name in buckets

# Create the bucket if it doesn't exist
if not bucket_exists(bucket_name):
    try:
        s3.create_bucket(Bucket=bucket_name)
        print(f"Created bucket: {bucket_name}")
    except Exception as e:
        print(f"Error creating bucket: {str(e)}")
else:
    print(f"Bucket {bucket_name} already exists")

Bucket hr-knowledge-base-v02-257269733378 already exists


In [20]:
# Create a sample HR policy markdown file
hr_policy_v02 = """ # Human Resources (HR) Policy
Preamble
The Human Resources (HR) Policy of [Organization Name] is designed to define the principles, processes, and standards that govern the management of human capital in the organization. This policy serves as a comprehensive guide to ensure fair, consistent, and legally compliant practices that promote a productive, respectful, and inclusive workplace.
The organization recognizes that employees are its most valuable asset and is committed to fostering a work environment that attracts, develops, and retains top talent while ensuring alignment with the organization’s mission, values, and strategic objectives.

Policy Point 1: Recruitment & Selection
1.1 Objective
To ensure a transparent, merit-based, and non-discriminatory recruitment process that attracts qualified candidates who align with organizational values and role requirements.

1.2 Scope
Applies to all permanent, contractual, part-time, and temporary hiring.

1.3 Guidelines
Workforce Planning: HR will conduct annual workforce planning with department heads to identify staffing needs.
Job Descriptions: Each role must have an up-to-date job description approved by HR.
Advertising Vacancies: Positions shall be advertised internally and externally to ensure equal opportunity.
Selection Process: Structured interviews, competency-based assessments, and background checks will be conducted.
Equal Opportunity: All recruitment will comply with anti-discrimination laws, ensuring no bias based on age, gender, race, religion, or disability.
Offer Letters: Shall be issued only after approvals from HR and relevant hiring managers.

Policy Point 2: Onboarding & Induction
2.1 Objective
To integrate new hires effectively into the organization's culture, processes, and performance expectations.

2.2 Process
Pre-Onboarding: HR will provide offer confirmation, joining instructions, and required documentation list.
Orientation Program: Includes introduction to company history, values, organizational structure, key policies, IT systems, and safety protocols.
Probationary Period: All new employees will undergo a probationary period (3-6 months) with performance reviews at 30, 60, and 90 days.
Buddy System: New hires will be assigned a workplace buddy for their first month to aid in cultural integration.

Policy Point 3: Employee Classification & Contracts
3.1 Classification Categories
Permanent Employees
Fixed-Term Contract Employees
Part-Time Employees
Interns & Trainees

3.2 Employment Contracts
Must clearly state role, remuneration, benefits, working hours, probation period, and termination clauses.
Contracts must comply with national labor laws.

Policy Point 4: Compensation & Benefits
4.1 Salary Structure
Based on job grade, market benchmarking, and internal equity.
Reviewed annually through performance appraisal and market adjustment.

4.2 Benefits
Health insurance coverage for employee and eligible dependents.
Retirement benefits as per statutory requirements.
Paid leave entitlements (annual, sick, maternity/paternity, compassionate).
Employee Assistance Program (EAP) for mental well-being.

4.3 Payroll
Salaries will be processed monthly, with payslips provided electronically.
Statutory deductions (tax, social security, provident fund) will be applied as per law.

Policy Point 5: Working Hours, Attendance & Leave
5.1 Working Hours
Standard workweek: [e.g., 40 hours, Monday-Friday, 9 AM-6 PM].
Flexible work arrangements subject to manager approval.

5.2 Attendance
Electronic attendance system; employees must clock in/out.
Uninformed absence beyond 3 days may result in disciplinary action.

5.3 Leave Policy
Annual Leave: [X] days per year.
Sick Leave: [X] days with medical certificate for absences exceeding 2 days.
Maternity/Paternity Leave: As per statutory law.
Unpaid Leave: Subject to management approval.

Policy Point 6: Performance Management
6.1 Objective
To establish a fair and consistent performance evaluation process that aligns individual goals with organizational strategy.

6.2 Process
Goal Setting: Annual goals agreed upon between employee and manager.
Mid-Year Review: Formal review of progress.
Annual Appraisal: Based on KPIs, competencies, and behavioral attributes.
Performance Improvement Plans (PIP): For underperforming employees with clear timelines and support mechanisms.

Policy Point 7: Learning & Development
7.1 Commitment
The organization will invest in employee development through structured training and learning opportunities.

7.2 Framework

Training Needs Analysis (TNA) conducted annually.
Internal & External Training programs provided.
Career Development Plans for high-potential employees.
Tuition Assistance for approved courses.

Policy Point 8: Employee Conduct & Discipline
8.1 Code of Conduct
Employees must act with integrity, respect, and professionalism.

8.2 Disciplinary Process
Verbal Warning → Written Warning → Final Warning → Termination (depending on severity).
Immediate dismissal for gross misconduct (e.g., fraud, violence, harassment).

8.3 Workplace Harassment Policy
Zero tolerance for harassment, bullying, or discrimination. Complaints will be investigated confidentially.

Policy Point 9: Health, Safety & Welfare
9.1 Health & Safety Commitment
Compliance with Occupational Health & Safety (OHS) laws.
Risk assessments conducted annually.
First aid facilities and emergency procedures in place.

9.2 Wellness Programs
Annual health check-ups.
Fitness subsidies.
Mental health awareness campaigns.

Policy Point 10: Exit & Termination
10.1 Voluntary Resignation
Notice period: [e.g., 1 month].
Exit interview to gather feedback.

10.2 Termination by Employer
Based on performance, redundancy, or misconduct.
Notice or payment in lieu of notice as per contract.

10.3 Final Settlement
Clearance of company property.
Settlement of outstanding salary, benefits, and leave encashment.

Implementation & Review
HR will review the policy annually.
Changes require approval from the HR Director and CEO.
"""

In [17]:
## Save the HR policy content to a markdown file in S3
with open('hr_policy_v02.md', 'w', encoding='utf-8') as file:
    file.write(hr_policy_v02)
# Upload the markdown file to the S3 bucket
try:    
    s3.upload_file('hr_policy_v02.md', bucket_name, 'hr_policy_v02.md')
    print(f"HR policy file uploaded to S3 bucket {bucket_name} successfully.")  
except Exception as e:
    print(f"Error uploading HR policy file to S3: {str(e)}")    
    

HR policy file uploaded to S3 bucket hr-knowledge-base-v02-257269733378 successfully.


In [34]:
import json
import boto3

# model_id = "amazon.titan-text-express-v1"  # Default
# model_id = "amazon.titan-embed-image-v1"  # Default
model_id = "amazon.titan-text-express-v1"
# model_id = "amazon.titan-embed-text-v1"
# model_id = "amazon.titan-text-premier-v1:0"  # Default
# model_id = "anthropic.claude-v2:1"  # Default
region = "us-east-1"  # Change this to your AWS region

def query_hr_knowledge_base(user_query, knowledge_base_id, model_id=model_id, region=region):
    """
    Query HR Knowledge Base and generate a response using a RAG Model.
    Args:
        user_query (str): The user's question.
        knowledge_base_id (str): ID of the knowledge base to query.
        model_id (str): The model ID to use.
        region (str): AWS region where Bedrock is deployed.
    """
    bedrock_runtime = boto3.client("bedrock-runtime", region_name=region)
    bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name=region)

    try:
        retrieve_response = bedrock_agent_runtime.retrieve(
            knowledgeBaseId=knowledge_base_id,
            retrievalQuery={'text': user_query},
            retrievalConfiguration={
                'vectorSearchConfiguration': {
                    'numberOfResults': 5  # Top 5 relevant results
                }
            }
        )

        # Extract retrieved information
        retrieved_passages = []
        for result in retrieve_response.get('retrievedResults', []):
            content = result.get('content', {}).get('text', '')
            source = "Unknown source"
            location = result.get('location', {})

            if location:
                if 's3location' in location:
                    source = location.get('s3location', {}).get('uri', source)
                elif 'type' in location:
                    source = location.get('type', source)

            score = result.get('score', 0)
            retrieved_passages.append({
                'content': content,
                'source': source,
                'relevance_score': score
            })

        # Prepare context
        context = "\n\n".join([f"Passage: {p['content']}" for p in retrieved_passages])
        if not context:
            context = "No relevant information found in the knowledge base."

        # Construct prompt
        prompt = f"""
        You are HR Manager who is helping employees to easily understand policies.  Use only the following information to answer the query.
        If you do not know the answer based on this information then say:
        'No I do not have definitive answer for this as of now' — don't make up an answer.

        Context:
        {context}

        Question: {user_query}

        Answer:
        """

        # Select payload based on model type
        if "anthropic" in model_id.lower():
            print("Inside Anthropic Model")
            payload = {
                "anthropic_version": "bedrock-2023-05-31",
                "max_tokens": 512,
                "temperature": 0.5,
                "messages": [{"role": "user", "content": prompt}]
            }
        # elif "amazon.titan" in model_id.lower():
        #     print("Inside Amazon Titan Model")
        #     payload = {
        #         "inputText": prompt,
        #         "textGenerationConfig": {
        #             "maxTokenCount": 512,
        #             "temperature": 0.6,
        #             "topP": 0.9
        #         }
        #     }
        elif "amazon.titan" in model_id.lower():
            print("Inside Amazon Titan Model")

            if "embed" in model_id.lower():
        # Embedding model payload (no textGenerationConfig)
                 payload = {
                "inputText": prompt
                }
            else:
             # Text generation model payload
                 payload = {
                   "inputText": prompt,
                   "textGenerationConfig": {
                   "maxTokenCount": 800,
                   "temperature": 0.8,
                   "topP": 0.7
            }
        }
    #----###
        elif "meta.llama" in model_id.lower():
            print("Inside Meta Llama Model")
            payload = {
                "inputText": prompt,
                "textGenerationConfig": {
                    "maxTokenCount": 512,
                    "temperature": 0.7,
                    "topP": 0.9
                }
            }
        else:
            raise ValueError(f"Unsupported model ID: {model_id}. Please use a supported model ID for RAG workflows.")

        # Call the model
        invoke_response = bedrock_runtime.invoke_model(
            modelId=model_id,
            body=json.dumps(payload),
            contentType="application/json",
            accept="application/json"
        )

        # Parse the response
        response_body = json.loads(invoke_response['body'].read().decode('utf-8'))

        if "anthropic" in model_id.lower():
            generated_answer = response_body.get('content', [{}])[0].get("text", "")
        elif "amazon.titan" in model_id.lower():
            generated_answer = response_body.get('results', [{}])[0].get("outputText", '')
        elif "meta.llama" in model_id.lower():
            generated_answer = response_body.get('generation', '')
        else:
            generated_answer = "Error: Could not parse response from the model."

        # Final return
        return {
            "query": user_query,
            "retrieved_passages": retrieved_passages,
            "generated_answer": generated_answer,
            "model_used": model_id
        }
    # Titan Embeddings G1 - Textv1.2
    except Exception as e:
        return {
            "error": str(e),
            "query": user_query,
            "knowledge_base_id": knowledge_base_id,
            "retrieved_passages": [],
            "model_id": model_id
        }


In [44]:
##Retrievel Example
# Define a sample knowledge base ID - This would be a real ID in a production scenario
sample_kb_id = "GIVL72SHRJ"  # Replace with your actual knowledge base ID
#sample_kb_id = "ZHJZPXTKEV"
# sample_kb_id = "KU4HIUJJNS"  
print(f"Model Id is : {model_id}")

# Test with a question that should be answerable from our knowledge base
# query = "What type of wellness programs are run by company ?"
# query = 'Who conducts annual workforce planning ?'
# query = "Who can avail Remote working benefit?"
# query = "When medical certificate is required ?"
query = "On what basis employee's compensation & benefits are revised or determined ?"
# query = "What is the health and safety policy of the company."
#query = "How frequently salary is paid ?"  # Uncomment to use this query
result = query_hr_knowledge_base(query, sample_kb_id, model_id)
if result is not None:
    print("\n" + "=="*30)
    print(f"Query: {result.get('query', '')}")
    if result.get('retrieved_passages'):
        print(f"\nRetrieved Information:\n{result['retrieved_passages'][0]['content']}")
    else:
        print("\nNo retrieved information found.")
    if 'generated_answer' in result:
        print(f"\nGenerated Answer:\n {result['generated_answer']}")
    elif 'error' in result:
        print(f"\nError:\n {result['error']}")
    else:
        print("\nNo answer generated.")




Model Id is : amazon.titan-text-express-v1
Inside Amazon Titan Model

Query: On what basis employee's compensation & benefits are revised or determined ?

No retrieved information found.

Generated Answer:
  It is determined by job evaluation.
